In [1]:
import random
from game import Game, Move, Player
from strategies.rl import Q_learing, CustomState,  get_coordinates

class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move


class MyPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move
    
class RLPlayer(Player):
    def __init__(self, steps, learning_rate, discount_factor) -> None:
        super().__init__()
        ql = Q_learing(steps, learning_rate, discount_factor)
        self.value_dictionary = ql.train()
        
    def make_move(self, game: Game) -> tuple[tuple[int, int], Move]:
        current_state = CustomState(get_coordinates(game.get_board()))
        list_action = sorted(self.value_dictionary[current_state], key=self.value_dictionary[current_state].get)
        '''
        for action in list_action:
            if action not in (current_state.x.union(current_state.o)):
                current_state.x.add(action)
                break
        '''

        from_pos, move = list_action[0]
        return from_pos, move
    
g = Game()

player1 = RLPlayer(100, 0.1, 0.7)
player2 = RandomPlayer()



  0%|          | 0/5000 [00:00<?, ?it/s]


KeyboardInterrupt



In [2]:
g.print()
winner = g.play(player1, player2)
g.print()
print(f"Winner: Player {winner}")

[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]


ValueError: too many values to unpack (expected 2)